In [35]:
import pandas as pd
import datetime
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline
scaler = MinMaxScaler()

In [36]:
data_times = pd.read_csv('score.csv')[['tanggal','arena_id','score']]

In [37]:
data_times['tanggal'] = pd.to_datetime(data_times['tanggal'])

In [38]:
data_times = data_times.loc[data_times.tanggal >= '2020-01-01']

In [39]:
#hapus yang score 0
index_0 = data_times[data_times['score']== 0].index
data_times.drop(index_0,inplace=True)

In [40]:
data_times = data_times.sort_values(by=['arena_id','tanggal'],ascending=False)

In [41]:
data_times = data_times
data_times['score_norm']=data_times.score.apply\
(lambda x: (x-data_times.score.min())/(data_times.score.max()-data_times.score.min()))

In [42]:
ones = np.ones(data_times.shape[0])
data_times.insert(loc=3,column='count',value=ones)
jumlah_arena = data_times[['arena_id','count']]
jumlah_arena=jumlah_arena.groupby(['arena_id']).sum()

In [43]:
jumlah_arena = jumlah_arena.sort_values(by=['count'],ascending=False)

In [44]:
jumlah_arena = jumlah_arena.loc[jumlah_arena['count']>=30]

In [45]:
jumlah_arena

,count
arena_id,
5539,71.0
5554,71.0
420,71.0
419,71.0
418,71.0
...,...
5143,31.0
5238,30.0
6066,30.0


In [ ]:
type(arena)

In [46]:
#list arena yang layak
list_arena = jumlah_arena.index.to_list()

In [47]:
data_times=data_times.loc[data_times['arena_id'].isin(list_arena)]

In [48]:
data_times.drop(['count'],axis=1,inplace=True)

In [49]:
data_times.drop(['tanggal'],axis=1,inplace=True)

In [50]:
data_times.reset_index(drop=True)

,arena_id,score,score_norm
0,6066,52,0.052041
1,6066,6,0.005102
2,6066,13,0.012245
3,6066,6,0.005102
4,6066,13,0.012245
...,...,...,...
5103,145,1,0.000000
5104,145,1,0.000000
5105,145,1,0.000000
5106,145,2,0.001020


In [51]:
def trim(df,col,col_value):
    df_o = pd.DataFrame(columns=['arena_id','score','score_norm']) #blank
    for col_v in col_value:
        df_x = df.loc[df[col]==col_v][:30]
        df_o = df_o.append(df_x,ignore_index=True)
    
    return df_o
        

In [52]:
data_times_trim = trim(data_times,'arena_id',list_arena)

In [53]:
series_time = data_times_trim['score']

In [54]:
series_time = series_time.values.reshape(len(list_arena),30)

In [55]:
data_times_clean = pd.DataFrame(data=series_time)

In [56]:
data_times_clean.insert(loc=0,column='arena_id',value=list_arena)
#data_times.insert(loc=3,column='count',value=ones)

In [57]:
data_times_clean.set_index(keys=['arena_id'],drop=True,inplace=True)

In [58]:
data_times_clean

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
arena_id,,,,,,,,,,,,,,,,,,,,,
5539,11,17,15,13,4,10,13,7,9,11,...,26,17,49,42,16,9,36,13,7,26
5554,8,9,16,11,4,14,8,10,9,23,...,22,21,65,55,14,12,25,8,11,20
420,27,34,14,24,20,25,20,23,23,39,...,24,384,82,65,28,23,37,57,276,26
419,57,62,67,47,54,51,67,70,71,111,...,72,86,343,214,87,71,102,45,56,95
418,317,66,66,58,53,64,61,426,52,132,...,55,89,225,195,68,72,457,45,57,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5143,1,1,1,1,1,1,1,1,1,2,...,1,2,2,2,1,1,2,2,2,2
5238,2,1,1,7,1,1,1,1,1,1,...,3,2,1,2,1,1,1,1,1,2
6066,52,6,13,6,13,18,24,24,31,37,...,1,12,24,33,31,24,1,39,18,114


In [59]:
X_t0_t13 = data_times_clean.iloc[:,np.arange(0,14)]  #X_train
y_t14 = pd.DataFrame(data_times_clean[14])                       #y_train
X_t15_t28 = data_times_clean.iloc[:,np.arange(15,29)]#X_test
y_t30 = pd.DataFrame(data_times_clean[29])                      #y_test

In [60]:
X_t0_t13.to_csv('X_train',index=False,header=False)
y_t14.to_csv('y_train',index=False,header=False)
X_t15_t28.to_csv('X_test',index=False,header=False)
y_t30.to_csv('y_test',index=False,header=False)